In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.datasets import imdb
import keras
from keras.utils import to_categorical
from sklearn.metrics import f1_score

from keras import backend as K
from keras.engine.topology import Layer, InputSpec



Using TensorFlow backend.


In [8]:

BASE_DIR = ''
GLOVE_DIR = '/home/abhinav/data/GLOVE_DATA/glove.6B'
TEXT_DATA_DIR = '/home/abhinav/data/full_text'
THIRD_MOD_DIR = "/home/abhinav/data/third_model/"
SECOND_MOD_DIR = "/home/abhinav/data/second_model/"
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 50000
EMBEDDING_DIM = 200
FIFTH_MOD_DIR = "/home/abhinav/data/fifth_model/"
MOD_DIR = FIFTH_MOD_DIR

num_words = MAX_NB_WORDS

In [9]:
print('Loading data...')
#[x_train, y_train] = pd.read_pickle(MOD_DIR + "train_data_10ratio") 
"""print(x_train.shape, 'train sequences')
print(y_train.shape, 'test sequences')
"""
[x_test, y_test] = pd.read_pickle(MOD_DIR + "test_dataset") 

[x_val, y_val ] = pd.read_pickle(MOD_DIR + "validation_dataset")

Loading data...


In [36]:
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D,merge, MaxPooling1D, Embedding,Merge,  Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers.merge import Add,Concatenate,Dot

In [7]:
embedding_matrix = pd.read_pickle("/home/abhinav/data/fifth_model/embedding_matrix")

In [3]:
class Metrics(keras.callbacks.Callback):
    def __init__(self,filepath):
        self.filepath = filepath
        self.best = -np.Inf
    def on_epoch_end(self, epoch,batch, logs={}):
        predict = self.model.predict(self.validation_data[0],batch_size = 512)
        predict = prob_to_label(predict)
        targ = self.validation_data[1]
        self.f1s=f1(targ, predict)
        """if self.f1s > self.best:
            self.best = self.f1s
            print('Epoch %05d: improved f1 to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, self.best, self.filepath))
            self.model.save(self.filepath,overwrite =True)"""
        print ("Weighted F1 score found on Validation dataset : " ,self.f1s)
        return

from sklearn.metrics import precision_recall_fscore_support
def f1(y_true, y_pred):
    return precision_recall_fscore_support(y_true = y_true, y_pred = y_pred)
ndm = keras.optimizers.Nadam(lr = 0.001)    

def prob_to_label(array):
    for a in array:
        if a[0] > a[1] :
            a[0] = 1
            a[1] = 0
        elif a[0] < a[1]:
            a[0] = 0
            a[1] = 1
        else:
            ind = np.random.randint(2)
            a[ind]= 1
            a[int(1-ind)] = 0
    return array

In [12]:
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model


In [21]:
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [22]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)

In [23]:
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',
              optimizer=ndm,
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1000, 200)         10000000  
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               240800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 10,241,202
Trainable params: 10,241,202
Non-trainable params: 0
_________________________________________________________________


In [24]:
initial_weights = model.get_weights()

In [25]:
print('Train...')
metrics = Metrics(MOD_DIR + "none")
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          class_weight = {0. : 1, 1. : 8},
validation_data=[x_test, y_test],
         callbacks = [metrics])

Train...
Train on 13150 samples, validate on 15165 samples
Epoch 1/4
13150/13150 [==============================] - 1533s - loss: 0.8992 - acc: 0.8192 - val_loss: 0.3532 - val_acc: 0.9022
Epoch 2/4
13150/13150 [==============================] - 1712s - loss: 0.7873 - acc: 0.8448 - val_loss: 0.3915 - val_acc: 0.8706
Epoch 3/4
13150/13150 [==============================] - 1613s - loss: 0.6343 - acc: 0.8599 - val_loss: 0.3398 - val_acc: 0.8578
Epoch 4/4
13150/13150 [==============================] - 1672s - loss: 0.3731 - acc: 0.9189 - val_loss: 0.2199 - val_acc: 0.9118


In [47]:
a = np.asarray([[0.4,0.6],[0.5,0.5],[0.46,0.54]])

a.shape

In [103]:
prob_to_label(a)

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [4]:
from keras.models import load_model
model1 = load_model("/home/abhinav/data/second_model/the_best_model_with_trained_word_embeddings")